In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [4]:
df.head(5)

,artist,song,text
0,Iggy Pop,I Wanna Be Your Dog,So messed up I want you here \r\nAnd in my ro...
1,Howard Jones,Bounce Right Back,I was walking down the street \r\nWith my old...
2,Jennifer Lopez,I Need Love,"Oh, oh oh... \r\n \r\nThis is where my story..."
3,You Am I,How Much Is Enough,Would it stain \r\nIf the lights fell down on...
4,Soundgarden,Loud Love,There's no time to keep it low \r\nI've been ...


In [5]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [6]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [7]:
import nltk
nltk.download('punkt')


[nltk_data] Downloading package punkt to C:\Users\Pragyan
[nltk_data]     Karki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [11]:
similarity[0]

array([1.        , 0.02856967, 0.07526671, ..., 0.00546118, 0.01097488,
       0.07693828])

In [12]:
df[df['song']== 'Waiting For The Man'].index[0]

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [ ]:
recommendation("Toyfriend")

In [ ]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))